In [1]:
# Define the AWS env variables if you are using AWS Auth:
%env AWS_REGION= region
%env AWS_ACCESS_KEY_ID= key
%env AWS_SECRET_ACCESS_KEY= secret

env: AWS_REGION=region
env: AWS_ACCESS_KEY_ID=key
env: AWS_SECRET_ACCESS_KEY=secret


In [2]:
import pyspark
from pyspark.sql import SparkSession
import os


conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
    
    # first we will define the packages that we need. Iceberg Spark runtime
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.2.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
        
    # This property allows us to add any extensions that we want to use
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
    
    # configures a new catalog to a particular implementation of SparkCatalog
        .set('spark.sql.catalog.glue', 'org.apache.iceberg.spark.SparkCatalog')
    
    # particular type of catalog we are using
        .set('spark.sql.catalog.glue.catalog-impl', 'org.apache.iceberg.aws.glue.GlueCatalog')
    
    # engine writes to the warehouse
        .set('spark.sql.catalog.glue.warehouse', 's3://my-bucket/warehouse/')
    
    # changes IO impl of catalog, mainly for changing writing data to object storage
        .set('spark.sql.catalog.glue.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/home/docker/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/docker/.ivy2/cache
The jars for the packages stored in: /home/docker/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fa930a13-7dc5-4267-8f33-fd7877a6eb80;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.2.0 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found software.amazon.awssdk#annotations;2.17.178 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found software.amazon.awssdk#http-client-spi;2.17.178 in central
	found software

23/08/21 19:14:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark Running


# Create Table ProductSales

In [8]:
# New:
spark.sql(
    """CREATE TABLE IF NOT EXISTS glue.test.productsales
            (OrderID string, Product string, Qty string, Price string, Date string, Address string) USING iceberg"""
)

DataFrame[]

In [9]:
spark.sql(
    """CREATE OR REPLACE TEMPORARY VIEW salesview USING csv
            OPTIONS (path "sales_clean_new.csv", header true)"""
)
spark.sql("INSERT INTO glue.test.productsales SELECT * FROM salesview")

DataFrame[]

In [10]:
spark.sql("SELECT * FROM glue.test.productsales").toPandas()

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


,OrderID,Product,Qty,Price,Date,Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
18261,194090,Google Phone,1,600,04/08/19 17:11,"177 Jackson St, Los Angeles, CA 90001"
18262,194091,AA Batteries (4-pack),1,3.84,04/15/19 16:02,"311 Forest St, Austin, TX 73301"
18263,194092,AAA Batteries (4-pack),2,2.99,04/28/19 14:36,"347 Sunset St, San Francisco, CA 94016"
18264,194093,AA Batteries (4-pack),1,3.84,04/14/19 15:09,"835 Lake St, Portland, OR 97035"


# CREATE A NEW BRANCH

In [11]:
spark.sql("ALTER TABLE glue.test.productsales CREATE BRANCH ETL_branch").show()

23/08/21 19:28:55 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up
++
||
++
++



# Check all the existing branches

In [12]:
spark.sql("SELECT * FROM glue.test.productsales.refs").toPandas()

,name,type,snapshot_id,max_reference_age_in_ms,min_snapshots_to_keep,max_snapshot_age_in_ms
0,ETL_branch,BRANCH,3416205539179840364,NaN,NaN,NaN
1,main,BRANCH,3416205539179840364,NaN,NaN,NaN


# Query from this new branch explicitly

In [13]:
spark.sql("SELECT * FROM glue.test.productsales VERSION AS OF 'ETL_branch'").toPandas()

,OrderID,Product,Qty,Price,Date,Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
18261,194090,Google Phone,1,600,04/08/19 17:11,"177 Jackson St, Los Angeles, CA 90001"
18262,194091,AA Batteries (4-pack),1,3.84,04/15/19 16:02,"311 Forest St, Austin, TX 73301"
18263,194092,AAA Batteries (4-pack),2,2.99,04/28/19 14:36,"347 Sunset St, San Francisco, CA 94016"
18264,194093,AA Batteries (4-pack),1,3.84,04/14/19 15:09,"835 Lake St, Portland, OR 97035"


# WRITE stage:

In [14]:
spark.sql(
    """ALTER TABLE glue.test.productsales
       SET TBLPROPERTIES ('write.wap.enabled'='true')"""
)

23/08/21 19:46:32 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


DataFrame[]

In [15]:
spark.conf.set('spark.wap.branch', 'ETL_branch')

# Ingest some new records

In [16]:
spark.sql(
    """CREATE OR REPLACE TEMPORARY VIEW salesetlnew USING csv
            OPTIONS (path "Sales_January_2019.csv", header true)"""
)

DataFrame[]

In [17]:
spark.sql("INSERT INTO glue.test.productsales SELECT * FROM salesetlnew")

DataFrame[]

# Query to see if the new records were ingested

In [19]:
spark.sql("SELECT * FROM glue.test.productsales VERSION AS OF 'ETL_branch'").toPandas()

,OrderID,Product,Qty,Price,Date,Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"
...,...,...,...,...,...,...
27984,194090,Google Phone,1,600,04/08/19 17:11,"177 Jackson St, Los Angeles, CA 90001"
27985,194091,AA Batteries (4-pack),1,3.84,04/15/19 16:02,"311 Forest St, Austin, TX 73301"
27986,194092,AAA Batteries (4-pack),2,2.99,04/28/19 14:36,"347 Sunset St, San Francisco, CA 94016"
27987,194093,AA Batteries (4-pack),1,3.84,04/14/19 15:09,"835 Lake St, Portland, OR 97035"


# Query the 'main' branch to validate

In [20]:
spark.sql("SELECT * FROM glue.test.productsales VERSION AS OF 'main'").toPandas()

,OrderID,Product,Qty,Price,Date,Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
18261,194090,Google Phone,1,600,04/08/19 17:11,"177 Jackson St, Los Angeles, CA 90001"
18262,194091,AA Batteries (4-pack),1,3.84,04/15/19 16:02,"311 Forest St, Austin, TX 73301"
18263,194092,AAA Batteries (4-pack),2,2.99,04/28/19 14:36,"347 Sunset St, San Francisco, CA 94016"
18264,194093,AA Batteries (4-pack),1,3.84,04/14/19 15:09,"835 Lake St, Portland, OR 97035"


# Audit Stage:

* NULL Value check

In [21]:
from pyspark.sql.functions import isnan, when, count, col

# Load your DataFrame
df_qc_sales = spark.read.option("branch", "ETL_branch") \
        .format("iceberg") \
        .load("glue.test.productsales") \

# Check for NULL values in all columns
null_counts_new = df_qc_sales.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_qc_sales.columns]).collect()[0]

# Check if any columns have NULL values
if any([null_counts_new[i] > 0 for i in range(len(null_counts_new))]):
    print("There are NULL values in the DataFrame.")
else:
    print("There are no NULL values in the DataFrame.")


There are NULL values in the DataFrame.


In [22]:
print(null_counts_new)

Row(OrderID=26, Product=26, Qty=26, Price=26, Date=26, Address=26)


* Duplicate records

In [34]:
from pyspark.sql import functions as F

# Assuming you've loaded your DataFrame as df_qc_sales

# Group by all columns and count occurrences of each combination, then filter those with a count greater than 1
duplicate_counts = df_qc_sales.groupBy(df_qc_sales.columns) \
                              .agg(F.count('*').alias('count')) \
                              .filter('count > 1')

# Check if there are any duplicate rows
if duplicate_counts.count() > 0:
    print("There are duplicate rows in the DataFrame")
else:
    print("There are no duplicate rows in the DataFrame")

# Join the duplicate_counts with the original DataFrame on all columns to get the actual records, then drop duplicates
duplicated_records = df_qc_sales.join(duplicate_counts.drop('count'), on=df_qc_sales.columns, how="inner") \
                                .dropDuplicates()

# Add the count column back to the dataframe
duplicated_records = duplicated_records.join(duplicate_counts, on=df_qc_sales.columns, how="left")

# Display duplicated records with their counts
duplicated_records.show()


There are duplicate rows in the DataFrame


+--------+--------------------+----------------+----------+--------------+--------------------+-----+
| OrderID|             Product|             Qty|     Price|          Date|             Address|count|
+--------+--------------------+----------------+----------+--------------+--------------------+-----+
|  142071|AA Batteries (4-p...|               1|      3.84|01/17/19 23:02|131 2nd St, Bosto...|    2|
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|   16|
|  145143|Lightning Chargin...|               1|     14.95|01/06/19 03:01|182 Jefferson St,...|    2|
|  146765|        Google Phone|               1|       600|01/21/19 11:23|918 Highland St, ...|    2|
|  147707|    Wired Headphones|               1|     11.99|01/04/19 16:50|883 4th St, Dalla...|    2|
|  148984|USB-C Charging Cable|               1|     11.95|01/08/19 17:36|562 14th St, Bost...|    2|
|  149149|Lightning Chargin...|               1|     14.95|01/12/19 12:30|180 1st 

* Date Consistency

In [38]:
from pyspark.sql.functions import unix_timestamp, col, lit, count

# Convert the date string into a DateType column
df_with_date = df_qc_sales.withColumn("parsed_date", unix_timestamp(col("Date"), "MM/dd/yy HH:mm").cast("timestamp"))

# Filter and group by invalid dates, then count occurrences
invalid_dates_df = df_with_date.filter(col("parsed_date").isNull()) \
                               .groupBy("Date").agg(count("*").alias("count"))

# Display the invalid dates and their counts
invalid_dates_count = invalid_dates_df.count()
if invalid_dates_count > 0:
    print(f"There are {invalid_dates_count} distinct invalid dates in the DataFrame.")
    invalid_dates_df.show()
else:
    print("All dates are valid in the DataFrame.")

# Define an expected date range (e.g., from 2019-01-01 to 2020-12-31)
start_date = "2019-01-01"
end_date = "2019-04-31"

# Check for dates outside the expected range
out_of_range_dates_count = df_with_date.filter((col("parsed_date") < lit(start_date)) | (col("parsed_date") > lit(end_date))).count()
if out_of_range_dates_count > 0:
    print(f"There are {out_of_range_dates_count} dates outside the expected range in the DataFrame.")
else:
    print(f"All dates are within the expected range ({start_date} to {end_date}) in the DataFrame.")


There are 2 distinct invalid dates in the DataFrame.


+----------+-----+
|      Date|count|
+----------+-----+
|Order Date|   16|
|      null|   26|
+----------+-----+



All dates are within the expected range (2019-01-01 to 2019-04-31) in the DataFrame.


# Applying Fixes to the Data Quality issues

In [39]:
from pyspark.sql.functions import col, coalesce, percentile_approx, lit, unix_timestamp, count

# Median Imputation
# --------------------------

# Calculate the median value for each column
medians = df_qc_sales.agg(*[percentile_approx(col(c), 0.5).alias(c) for c in df_qc_sales.columns])

# Replace null values with the calculated median
df_imputed = df_qc_sales.select([coalesce(col(c), lit(str(medians.first()[c]))).alias(c) for c in df_qc_sales.columns])

# Show the imputed DataFrame
df_imputed.show()

# Removing Duplicate Records
# --------------------------

df_no_duplicates = df_imputed.dropDuplicates()

# Invalid Date Removal
# --------------------------

# Convert the date string into a DateType column.
# If the date string is invalid, this conversion will result in null values.
df_with_date = df_no_duplicates.withColumn("parsed_date", unix_timestamp(col("Date"), "MM/dd/yy HH:mm").cast("timestamp"))

# Remove rows with invalid dates
df_valid_dates = df_with_date.filter(col("parsed_date").isNotNull())

# Final DataFrame
# --------------------------

# If you'd like to drop the "parsed_date" column added for validation:
df_final = df_valid_dates.drop("parsed_date")

df_final.show()


+-------+--------------------+---+------+--------------+--------------------+
|OrderID|             Product|Qty| Price|          Date|             Address|
+-------+--------------------+---+------+--------------+--------------------+
| 176558|USB-C Charging Cable|  2| 11.95|04/19/19 08:46|917 1st St, Dalla...|
| 176559|Bose SoundSport H...|  1| 99.99|04/07/19 22:30|682 Chestnut St, ...|
| 176560|        Google Phone|  1|   600|04/12/19 14:38|669 Spruce St, Lo...|
| 176560|    Wired Headphones|  1| 11.99|04/12/19 14:38|669 Spruce St, Lo...|
| 176561|    Wired Headphones|  1| 11.99|04/30/19 09:27|333 8th St, Los A...|
| 176562|USB-C Charging Cable|  1| 11.95|04/29/19 13:03|381 Wilson St, Sa...|
| 176563|Bose SoundSport H...|  1| 99.99|04/02/19 07:46|668 Center St, Se...|
| 176564|USB-C Charging Cable|  1| 11.95|04/12/19 10:58|790 Ridge St, Atl...|
| 176565|  Macbook Pro Laptop|  1|  1700|04/24/19 10:38|915 Willow St, Sa...|
| 176566|    Wired Headphones|  1| 11.99|04/08/19 14:05|83 7th S

+-------+--------------------+---+------+--------------+--------------------+
|OrderID|             Product|Qty| Price|          Date|             Address|
+-------+--------------------+---+------+--------------+--------------------+
| 141234|              iPhone|  1|   700|01/22/19 21:25|944 Walnut St, Bo...|
| 141235|Lightning Chargin...|  1| 14.95|01/28/19 14:15|185 Maple St, Por...|
| 141236|    Wired Headphones|  2| 11.99|01/17/19 13:33|538 Adams St, San...|
| 141237|    27in FHD Monitor|  1|149.99|01/05/19 20:33|738 10th St, Los ...|
| 141238|    Wired Headphones|  1| 11.99|01/25/19 11:59|387 10th St, Aust...|
| 141239|AAA Batteries (4-...|  1|  2.99|01/29/19 20:22|775 Willow St, Sa...|
| 141240|27in 4K Gaming Mo...|  1|389.99|01/26/19 12:16|979 Park St, Los ...|
| 141241|USB-C Charging Cable|  1| 11.95|01/05/19 12:04|181 6th St, San F...|
| 141242|Bose SoundSport H...|  1| 99.99|01/01/19 10:30|867 Willow St, Lo...|
| 141243|Apple Airpods Hea...|  1|   150|01/22/19 21:20|657 John

In [43]:
df_final.count()

27937

In [60]:
spark.sql("SELECT * FROM glue.test.productsales VERSION AS OF 'ETL_branch'").toPandas()

,OrderID,Product,Qty,Price,Date,Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
27932,150497,20in Monitor,1,109.99,01/26/19 19:09,"95 8th St, Dallas, TX 75001"
27933,150498,27in FHD Monitor,1,149.99,01/10/19 22:58,"403 7th St, San Francisco, CA 94016"
27934,150499,ThinkPad Laptop,1,999.99,01/21/19 14:31,"214 Main St, Portland, OR 97035"
27935,150500,AAA Batteries (4-pack),2,2.99,01/15/19 14:21,"810 2nd St, Los Angeles, CA 90001"


In [47]:
spark.sql("SELECT * FROM glue.test.productsales VERSION AS OF 'main'").toPandas()

,OrderID,Product,Qty,Price,Date,Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
18261,194090,Google Phone,1,600,04/08/19 17:11,"177 Jackson St, Los Angeles, CA 90001"
18262,194091,AA Batteries (4-pack),1,3.84,04/15/19 16:02,"311 Forest St, Austin, TX 73301"
18263,194092,AAA Batteries (4-pack),2,2.99,04/28/19 14:36,"347 Sunset St, San Francisco, CA 94016"
18264,194093,AA Batteries (4-pack),1,3.84,04/14/19 15:09,"835 Lake St, Portland, OR 97035"


# Publish Stage:

In [61]:
spark.sql("SELECT * FROM glue.test.productsales.refs").toPandas()

,name,type,snapshot_id,max_reference_age_in_ms,min_snapshots_to_keep,max_snapshot_age_in_ms
0,ETL_branch,BRANCH,6348938829066125512,NaN,NaN,NaN
1,main,BRANCH,3416205539179840364,NaN,NaN,NaN


In [ ]:
spark.sql("CALL glue.system.cherrypick_snapshot('test.productsales', 6348938829066125512)").show()

In [ ]:
spark.sql("SELECT * FROM glue.test.productsales VERSION AS OF 'main'").toPandas()

# Unset the WAP property

In [ ]:
spark.conf.unset('spark.wap.branch')

# If data quality isn't good as per the set expectations, just DROP the branch

In [ ]:
spark.sql("ALTER TABLE glue.test.productsales DROP BRANCH ETL_branch")